In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp

In [ ]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

# Directory paths
base_dir = '../data/HaGRID/modified/images'
landmark_file = '../data/HaGRID/modified/csv/hand_landmarks_hagrid_modified.csv'

# Number of images to process in each batch
batch_size = 100 

In [ ]:
# Prepare a list to store landmarks
landmarks_list = []

# Column names
columns = ['image_path', 'label', 'handedness'] + [f'{axis}_{i}' for i in range(21) for axis in ['x', 'y', 'z']]

# Function to save batch of landmarks to CSV
def save_batch(landmarks_list, batch_num):
    landmarks_df = pd.DataFrame(landmarks_list, columns=columns)
    if batch_num == 0:
        landmarks_df.to_csv(landmark_file, index=False, mode='w')  # Overwrite on first batch
    else:
        landmarks_df.to_csv(landmark_file, index=False, mode='a', header=False)  # Append for subsequent batches

In [ ]:
batch_num = 0
for label in os.listdir(base_dir):
    label_dir = os.path.join(base_dir, label)
    if os.path.isdir(label_dir):
        print(f'Processing directory: {label_dir}')
        for filename in os.listdir(label_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(label_dir, filename)
                image = cv2.imread(image_path)
                
                if image is not None:
                    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    # Process the image and detect hand landmarks
                    result = hands.process(image_rgb)

                    if result.multi_hand_landmarks and len(result.multi_hand_landmarks) == 1:
                        hand_landmarks = result.multi_hand_landmarks[0]
                        landmarks = [image_path, label]
                        
                        # Add hand label (left or right)
                        handedness = result.multi_handedness[0].classification[0].label
                        landmarks.append(handedness)
                        
                        # Add landmark coordinates
                        landmarks += [coord for lm in hand_landmarks.landmark for coord in (lm.x, lm.y, lm.z)]
                        
                        landmarks_list.append(landmarks)
                        print(f'Landmarks detected for image: {image_path}')
                    else:
                        if result.multi_hand_landmarks:
                            print(f'Two hands detected, ignoring image: {image_path}')
                        else:
                            print(f'No landmarks detected for image: {image_path}')
                else:
                    print(f'Failed to read image: {image_path}')
                
                # Save batch if size is reached
                if len(landmarks_list) >= batch_size:
                    save_batch(landmarks_list, batch_num)
                    batch_num += 1
                    print(f'Batch Number: {batch_num}')
                    landmarks_list = []

# Save any remaining landmarks
if landmarks_list:
    save_batch(landmarks_list, batch_num)

In [ ]:
# Release MediaPipe resources
hands.close()

print(f'Finished processing.')